# Batch Correction with ComBat
This notebook takes TPM of samples with batch effects and removes them with ComBat package

It is used and shared as part of the publication:

    Characterization of CA-MRSA TCH1516 exposed to Nafcillin in 
    Bacteriological and Physiological Media

In [1]:
suppressPackageStartupMessages(library('sva'))
suppressPackageStartupMessages(library('limma'))

# Load and QC
- Load the TPM data
- Remove any points with N/A and genes whose expression level are low across all conditions

In [3]:
data <- read.csv('pub_tpm.csv', row.names='', check.names=FALSE)
data <- log2(data + 1)
data <- data[complete.cases(data),]
cat('Number of Genes:', nrow(data), '\n', 'Number of Samples:', ncol(data))

Number of Genes: 2486 
 Number of Samples: 18

# calc_combat 
Takes in `data` (log TPM), subsets based on `condition` and runs ComBat

Returns ComBat adjusted log TPM values

In [4]:
calc_combat <- function(data, condition) {
    conc <- c()
    sample <- c()
    batch <- c()
    media <- c()
    for (i in colnames(data)) {
        # get all CAMHB
        if (grepl(condition, i)) {
            names <- strsplit(i, ' ')[[1]]
            conc <- c(conc, names[2])
            media <- c(media, names[1])
            batch <- c(batch,names[4])
            sample <- c(sample, i[1])
        }
    }

    df <- data.frame(Sample<-sample,
                   Naf<-conc,
                   Batch<-batch,
                   Media<-media)
    df$Interaction <- factor(paste0(df$Media, df$Naf))

    mod <- model.matrix(~as.factor(Interaction), data=df)
    combat <- ComBat(dat=as.matrix(data[,colnames(data) %in% df$Sample]), 
                    batch=df$Batch, mod=mod,
                   par.prior=TRUE, prior.plots=FALSE)
    }

In [5]:
#remove batch effect from each of the dataset
combat_r10lb <- calc_combat(data, 'R10LB')
combat_camhb <- calc_combat(data, 'CAMHB')

Found3batches
Adjusting for2covariate(s) or covariate level(s)


Standardizing Data across genes


Fitting L/S model and finding priors
Finding parametric adjustments
Adjusting the Data

Found3batches
Adjusting for2covariate(s) or covariate level(s)


Standardizing Data across genes


Fitting L/S model and finding priors
Finding parametric adjustments
Adjusting the Data



In [6]:
#merge the two datasets
combat_tpm <- merge(combat_camhb, combat_r10lb, by=0)
rownames(combat_tpm) <- combat_tpm$Row.names
combat_tpm <- combat_tpm[,!colnames(combat_tpm) %in% c('Row.names')]

In [10]:
write.csv(combat_tpm, 'combat_tpm.csv')

In [9]:
sessionInfo()

R version 3.5.1 (2018-07-02)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 16.04.5 LTS

Matrix products: default
BLAS: /home/saugat/anaconda3/envs/RNAseqPipeline/lib/R/lib/libRblas.so
LAPACK: /home/saugat/anaconda3/envs/RNAseqPipeline/lib/R/lib/libRlapack.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] limma_3.36.3        sva_3.28.0          BiocParallel_1.14.2
[4] genefilter_1.62.0   mgcv_1.8-24         nlme_3.1-137       
[7] jsonlite_1.5        formatR_1.5        

loaded via a namespace (and not attached):
 [1] Rcpp_0.12.18     